# Ising-Modell

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird das Ising-Modell in ein und zwei Dimensionen mit zwei verschiedenen Algorithmen untersucht, dem Metropolis- und dem Wolff-Algorithmus. Das Ziel besteht darin, die spezifische Wärme und die magnetische Suszeptibilität als Funktion der Temperatur zu bestimmen. Auf dem Weg dorthin werden einige Zwischenschritte eingelegt. Zunächst wird die Ausrichtung der einzelnen Spins für ein eindimensionales Ising-System als Funktion der Iterationsschritte bestimmt und graphisch dargestellt.
In den weiteren Schritten werden physikalische Größen des Gesamtsystems betrachtet. Zunächst wird die Entwicklung der Energie und der Magnetisierung als Funktion der Iterationsschritte berechnet. Anschließend erfolgt nach einer Initialisierungsphase die Mittelung dieser beiden Größen, und die Mittelwerte werden wiederum als Funktion der Iterationsschritte dargestellt, so dass eine Beurteilung der Konvergenz möglich wird. Im Folgenden wird eine entsprechende Rechnung für die spezifische Wärme und die magnetische Suszeptibilität durchgeführt. Abschließend werden diese beiden Größen dann als Funktion der Temperatur bestimmt.

## Importanweisungen

Hier importieren wir zum ersten Mal das `defaultdict` aus dem `collections`-Modul der Python-Standardbibliothek. In der Funktion `thermo_values_development` werden wir das `defaultdict` verwenden, damit neue Einträge in das Dictionary automatisch mit einer leeren Liste vorbelegt werden, so dass direkt Elemente an diese Liste angehängt werden können.

In [1]:
from collections import defaultdict, namedtuple
from math import exp
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

plt.style.use("numphyspy.style")

## Darstellung des Zustands als Funktion der Iterationsschritte

### Bestimmung der nächsten Nachbarn

Für ein Gitter der Dimension `dimension` und der Anzahl `size` von Gitterplätzen je Raumrichtung mit periodischen Randbedingungen werden die Indizes der Gitterplätze der nächsten Nachbarn bestimmt. Dazu wird ein Array, das die Indizes der Gitterplätze enthält, in jede Raumrichtung um einen Gitterplatz vorwärts und rückwärts verschoben. Hieraus ergibt sich ein zweidimensionales Array `neighbour`, das in der $n$-ten Zeile die Indizes der nächsten Nachbarn enthält.

In [2]:
def neighbours(size, dimension):
    neighbour = np.empty((size**dimension, 2*dimension),
                         dtype=int)
    index_shape = (size,)*dimension
    index = np.arange(size**dimension).reshape(index_shape)
    column = 0
    for axis in range(dimension):
        for shift in (-1, 1):
            neighbour[:, column] = np.roll(index, shift,
                                           axis=axis
                                           ).flatten()
            column = column + 1
    return neighbour

### Erzeugung des Anfangszustands

Im Anfangszustand sind die Spins zufällig in eine der beiden möglichen Richtungs orientiert. Dieser Zustand entspricht einer unendlichen Temperatur. Um die Ergebnisse reproduzierbar zu machen, wird bei der Initialisierung des Zufallszahlengenerators ein `seed` gesetzt, das bei Bedarf abgeändert oder entfernt werden kann. Die Funktion `initial_state` gibt neben dem Zustand selbst auch dessen Energie und Magnetisierung zurück.

In [ ]:
rng = np.random.default_rng(123456)

def initial_state(n_spins, neighbour):
    state = rng.choice((-1, 1), size=n_spins)
    magnetization = np.sum(state)
    sum_nbr_spins = np.sum(state[neighbour], axis=1)
    energy = -np.sum(state*sum_nbr_spins)/2
    return state, energy, magnetization

### Iterationsschritt im Metropolis-Algorithmus

Der Metropolis-Algorithmus wird hier als Generatorfunktion implementiert. Nachdem diese Funktion einmal initialisiert wurde, können sukzessive Iterationsschritte durchgeführt werden, ohne immer wieder den Spinzustand für den nächsten Iterationsschritt übergeben zu müssen. Neben der Anzahl `size` der Gitterplätze je Raumrichtung und der Dimension `dimension` des Gitters muss auch die inverse Temperatur `beta` als Argument übergeben werden. Möchte man neben der Energie und der Magnetisierung auch den Spinzustand als Resultat erhalten, muss man das Argument `full` auf `True` setzen.

Zur Ausführung eines Iterationsschritts wird zufällig ein Spin ausgewählt und die Energieänderung bestimmt, die sich beim Umklappen des Spins ergeben würde. Erfüllt die Energieänderung die Bedingung des Metropolis-Algorithmus, so wird der Spin umgeklappt, und es werden die Werte für die Energie und die Magnetisierung aktualisiert. Anschließend werden die Ergebnisse zurückgegeben und die Generatorfunktion wartet auf die Anforderung des nächsten Iterationsschritts.

In [ ]:
def metropolis_generator(size, dimension, beta, full=False):
    n_spins = size**dimension
    neighbour = neighbours(size, dimension)
    state, energy, magnetization = initial_state(n_spins,
                                                 neighbour)
    while True:
        if full:
            yield state, energy, magnetization
        else:
            yield energy, magnetization
        n_spin = rng.choice(n_spins)
        sum_nbr_spins = np.sum(state[neighbour[n_spin, :]])
        delta_e = 2*state[n_spin]*sum_nbr_spins
        if (delta_e < 0
                or rng.uniform() < exp(-beta*delta_e)):
            state[n_spin] = -state[n_spin]
            energy = energy + delta_e
            magnetization = magnetization + 2*state[n_spin]

### Iterationsschritt im Wolff-Algorithmus

Auch der Wolff-Algorithmus ist als Generatorfunktion implementiert. Sieht man vom zentralen Teil des Iterationsschritts selbst ab, ist die vorige Beschreibung der Implementation des Metropolis-Algorithmus auch hier anwendbar.

Der eigentlich Wolff-Iterationsschritt beginnt mit der zufälligen Auswahl eines Spins sowie der Definition von zwei Sets. `to_do` enthält die noch abzuarbeitenden Gitterplätze und `cluster` enthält die Gitterplätze, die zum zu invertierenden Cluster gehören. Der ausgewählte Spin wird gleich invertiert. Anschließend werden alle im Set `to_do` vorhandenen Gitterplätze abgearbeitet. Dazu wird einer der Gitterplätze ausgewählt und dessen nächste Nachbarn betrachtet. Wenn die Bedingung des Wolff-Algorithmus erfüllt ist, wird der entsprechende Nachbarplatz den Sets `to_do` und `cluster` hinzugefügt und der zugehörige Spin umgeklappt. Anschließend werden die Werte der Magnetisierung sowie der Energie aktualisiert.

In [ ]:
def wolff_generator(size, dimension, beta, full=False):
    n_spins = size**dimension
    neighbour = neighbours(size, dimension)
    p_limit = 1-exp(-2*beta)
    state, energy, magnetization = initial_state(n_spins,
                                                 neighbour)
    while True:
        if full:
            yield state, energy, magnetization
        else:
            yield energy, magnetization
        n_start = rng.choice(n_spins)
        to_do = {n_start}
        cluster = {n_start}
        state[n_start] = -state[n_start]
        while to_do:
            n_spin = to_do.pop()
            for nbr in neighbour[n_spin, :]:
                if (state[nbr] != state[n_spin]
                        and np.random.uniform() < p_limit):
                    to_do.add(nbr)
                    cluster.add(nbr)
                    state[nbr] = -state[nbr]
        delta_m = 2*state[n_start]*len(cluster)
        magnetization = magnetization + delta_m
        delta_e = 0
        for n_spin in cluster:
            sum_nbr_spins = 0
            for n_nbr_spin in neighbour[n_spin, :]:
                if n_nbr_spin not in cluster:
                    sum_nbr_spins += state[n_nbr_spin]
            delta_e = delta_e - 2*state[n_spin]*sum_nbr_spins
        energy = energy + delta_e

### Entwicklung des Spinzustands

Hier wird die Entwicklung des Spinzustands über `n_steps` Iterationsschritte mit Hilfe des Algorithmus `algorithm` berechnet. Auch wenn die Funktion `state_evolution` im Prinzip auch für höherdimensionale Gitter verwendbar ist, wird sie im Folgenden nur zur Berechnung der Entwicklung des Zustands eines eindimensionalen Ising-Gitters herangezogen.

In [ ]:
def state_evolution(size, dimension, beta, algorithm,
                    n_steps):
    state_sequence = np.empty((size, n_steps))
    for n, (state, energy, magnetization) in zip(
            range(n_steps),
            algorithm(size, dimension, beta, full=True)
    ):
        state_sequence[:, n] = state
    return state_sequence

### Implementierung der Bedienelemente

Um die Einstellungsmöglichkeiten in den folgenden Codezellen voneinander zu entkoppeln, wird hier eine Funktion definiert, die alle benötigen Widgets bereitstellt. Die tatsächlich verwendeten Bedienelemente werden an den betreffenden Stellen dokumentiert. Da die sinnvoll wählbaren Systemgrößen von der Gitterdimension abhängen, werden die auswählbaren Systemgrößen in Abhängigkeit von der gewählten Gitterdimension in `handle_dim_change` umgestellt.

In [ ]:
algorithms = {"Metropolis": metropolis_generator,
              "Wolff": wolff_generator}

def get_widget_dict():
    wide_label = {"description_width": "initial"}
    widget_dict = {"size":
                   widgets.IntSlider(
                       value=100, min=100, max=1000,
                       step=100,
                       description=r"$n_\text{spins}$"),
                   "dimension":
                   widgets.RadioButtons(
                       options=[("eindimensional", 1),
                                ("zweidimensional", 2)],
                       value=1, description="Dimension"),
                   "algorithm":
                   widgets.RadioButtons(
                       value="Metropolis",
                       options=("Metropolis", "Wolff"),
                       description="Algorithmus"),
                   "absmagn":
                   widgets.RadioButtons(
                       options=[("M", False),
                                ("|M|", True)],
                       value=False,
                       description="Magnetisierung"),
                   "temperature":
                   widgets.FloatSlider(
                       value=1, min=0.05, max=5,
                       step=0.05,
                       description="$T$"),
                   "temperature_range":
                   widgets.FloatRangeSlider(
                       value=(0.2, 5), min=0.05, max=5,
                       step=0.05,
                       description="Temperaturbereich",
                       style=wide_label),
                   "n_T":
                   widgets.IntSlider(
                       value=30, min=10, max=200, step=10,
                       description="Temperaturwerte",
                       style=wide_label),
                   "log_n_steps_init":
                   widgets.IntSlider(
                       value=3, min=1, max=7, step=1,
                       description=r"$\log_{10}"
                                   r"(n_\text{init})$"),
                   "log_n_steps":
                   widgets.IntSlider(
                       value=4, min=3, max=9, step=1,
                       description=r"$\log_{10}"
                                   r"(n_\text{steps})$"),
                   "log_n_steps_min_max":
                   widgets.IntRangeSlider(
                       value=(0, 5), min=0, max=9,
                       description=r"$\log_{10}"
                                   r"(n_\text{steps})$"),
                   "n_steps":
                   widgets.IntSlider(
                       value=100, min=50, max=1000, step=50,
                       description=r"$n_\text{steps}$")
                   }

    def handle_dim_change(change):
        attributes = {1: {"max": 1000, "min": 50, "step": 50,
                          "value": 100},
                      2: {"step": 5, "min": 5, "max": 40,
                          "value": 20}}
        for k, v in attributes[change.new].items():
            setattr(widget_dict["size"], k, v)

    widget_dict["dimension"].observe(handle_dim_change,
                                     names="value")
    return widget_dict

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

### Graphische Darstellung des Zustands eines eindimensionalen Ising-Systems als Funktion der Iterationsschritte

Mit Hilfe der Bedienelemente können die folgenden Parameter eingestellt werden:
- `algorithm`: zu verwendender Algorithmus (Metropolis oder Wolff)
- `temperature`: dimensionslose Temperatur
- `size`: Anzahl der Gitterplätze je Raumdimension
- `n_steps`: Anzahl der Iterationsschritte

Die Dimension des Gitters ist hier auf 1 festgelegt, da nur dann eine graphische Darstellung des gesamten Spinzustands als Funktion der Iterationsschritte möglich ist. Die zwei möglichen Spinzustände werden in unterschiedlichen Farben dargestellt. Horizontal läuft die Zeit und vertikal ist das eindimensionale Gitter dargestellt.

In [ ]:
@interact(**get_widget_dict())
def plot_configs(algorithm, temperature, size, n_steps):
    dimension = 1
    beta = 1 / temperature
    state_sequence = state_evolution(size, dimension, beta,
                                     algorithms[algorithm],
                                     n_steps)

    fig, ax = plt.subplots()
    ax.imshow(state_sequence, aspect="auto",
              interpolation="none")
    ax.set_xlabel("$t$")
    ax.set_ylabel("$x$")

## Energie und Magnetisierung als Funktion der Iterationsschritte

### Energie, Magnetisierung sowie deren erste und zweite Momente

Neben der Energie und der Magnetisierung eines Spinzustands werden hier im Verlauf der Iterationen auch die Energie, die Magnetisierung sowie deren Quadrate aufsummiert und gemittelt. Die so bestimmten ersten und zweiten Momente werden später zur Berechnung der spezifischen Wärme und der magnetischen Suszeptibilität benötigt. Bevor die Mittelung beginnt; werden zuächst `n_init` Initialisierungsschritte durchgeführt. Anschließend werden laufende Mittelwerte berechnet bis `n_iters` Iterationsschritte erreicht wurden. Der Parameter `absmagn` erwartet einen Wahrheitswert, die bestimmt, ob der Betrag der Magnetisierung oder die Magnetisierung selbst gemittelt wird. Die Ergebnisse werden in einem *named tuple* zurückgegeben, so dass auf die einzelnen Elemente über entsprechende Attributnamen zugegriffen werden kann.

In [ ]:
thermo_values_keys = ("n_steps", "energy", "magnetization",
                      "e_sum", "e2_sum", "m_sum", "m2_sum")
ThermoValues = namedtuple("ThermoValues",
                          thermo_values_keys)

def thermo_values_development(size, dimension, beta,
                              algorithm, n_init, n_iters,
                              absmagn):
    state = algorithms[algorithm](size, dimension, beta)
    for n in range(n_init):
        next(state)
    thermo_values = defaultdict(list)
    n_steps = 0
    e_sum = 0
    e2_sum = 0
    m_sum = 0
    m2_sum = 0
    for n_todo in n_iters:
        for n in range(n_todo):
            energy, magnetization = next(state)
            e_sum = e_sum + energy
            e2_sum = e2_sum + energy**2
            if absmagn:
                magnetization = abs(magnetization)
            m_sum = m_sum + magnetization
            m2_sum = m2_sum + magnetization**2
        n_steps = n_steps + n_todo
        for k in thermo_values_keys:
            value = locals()[k]
            if k in ("e_sum", "e2_sum", "m_sum", "m2_sum"):
                value = value / n_steps
            thermo_values[k].append(value)
    for k in thermo_values_keys:
        thermo_values[k] = np.array(thermo_values[k])
    return ThermoValues(**thermo_values)

### Iteration auf einer logarithmischen Skala

Um eine äquidistante Darstellung der Resultate auf einer logarithmischen Skala zu erreichen, wird eine Generatorfunktion definiert, die die Anzahl der Iterationsschritte bis zum nächsten Datenpunkt zurückgibt. Es sollen dabei `n_values` Datenpunkte berechnet werden, wobei die Zehnerlogarithmen der Iterationen zwischen `log_n_steps_min` und `log_n_steps_max` liegen.

In [ ]:
def logarithmic_niters(log_n_steps_min, log_n_steps_max,
                       n_values):
    n_done = 0
    for n in np.logspace(log_n_steps_min, log_n_steps_max,
                         n_values, dtype=int):
        if n_done < n:
            yield n-n_done
            n_done = n

### Graphische Darstellung der Energie und der Magnetisierung als Funktion der Iterationsschritte

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `size`: Anzahl der Gitterplätze je Raumdimension
- `dimension`: Dimension des Gitters
- `temperature`: dimensionslose Temperatur
- `algorithm`: zu verwendender Algorithmus (Metropolis oder Wolff)
- `log_n_steps_min_max`: Endpunkte der Zehnerlogarithmen der Anzahl der Iterationsschritte, für die Daten dargestellt werden sollen

Es werden die Energie und die Magnetisierung je Spin als Funktion der Iterationsschritte graphisch dargestellt.

In [ ]:
@interact_start(**get_widget_dict())
def plot_e_m(size, dimension, temperature,
             algorithm, log_n_steps_min_max):
    log_n_steps_min, log_n_steps_max = log_n_steps_min_max
    n_spins = size**dimension
    n_iters = logarithmic_niters(log_n_steps_min,
                                 log_n_steps_max, 200)
    n_steps_init = 0
    beta = 1 / temperature
    absmagn = False
    thermo_values = thermo_values_development(
        size, dimension, beta, algorithm, n_steps_init,
        n_iters, absmagn)
    mean_energy = np.array(thermo_values.energy) / n_spins
    mean_magnetization = np.array(
        thermo_values.magnetization) / n_spins

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.semilogx(thermo_values.n_steps, mean_energy)
    ax1.set_xlabel("$n$")
    ax1.set_ylabel("$E/N$")

    ax2.semilogx(thermo_values.n_steps, mean_magnetization)
    ax2.set_xlabel("$n$")
    ax2.set_ylabel("$M/N$")

## Graphische Darstellung der mittleren Energie und der mittleren Magnetisierung als Funktion der Iterationsschritte

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `size`: Anzahl der Gitterplätze je Raumdimension
- `dimension`: Dimension des Gitters
- `temperature`: dimensionslose Temperatur
- `algorithm`: zu verwendender Algorithmus (Metropolis oder Wolff)
- `log_n_steps_min_max`: Endpunkte der Zehnerlogarithmen der Anzahl der Iterationsschritte, für die Daten dargestellt werden sollen
- `absmagn`: falls gleich `True` wird statt der Magnetisierung deren Absolutbetrag zur Berechnung herangezogen

Da `n_steps_init` fest gleich null gesetzt wird, beginnt die Mittelung schon mit dem ersten Iterationsschritt. Bei Bedarf kann der Code angepasst werden, um eine Initialisierungsphase zu erreichen.

Es werden die mittlere Energie und die mittlere Magnetisierung als Funktion der Iterationsschritte graphisch dargestellt. 

In [ ]:
@interact_start(**get_widget_dict())
def plot_e_m_means(size, dimension, temperature, algorithm,
                   log_n_steps_min_max, absmagn):
    log_n_steps_min, log_n_steps_max = log_n_steps_min_max
    n_spins = size**dimension
    n_iters = logarithmic_niters(log_n_steps_min,
                                 log_n_steps_max, 200)
    n_steps_init = 0
    beta = 1 / temperature
    thermo_values = thermo_values_development(
        size, dimension, beta, algorithm, n_steps_init,
        n_iters, absmagn)
    mean_energy = thermo_values.e_sum / n_spins
    mean_magnetization = thermo_values.m_sum / n_spins

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.semilogx(thermo_values.n_steps, mean_energy)
    ax1.set_xlabel("$n$")
    ax1.set_ylabel(r"$\langle E \rangle/N$")

    if absmagn:
        label_m = r"$\langle \vert M\vert \rangle/N$"
        lower_m = 0
    else:
        label_m = r"$\langle M \rangle/N$"
        lower_m = -1
    ax2.semilogx(thermo_values.n_steps, mean_magnetization)
    ax2.set_xlabel("$n$")
    ax2.set_ylabel(label_m)
    ax2.set_ylim((lower_m, 1))

## Graphische Darstellung der spezifischen Wärme und der magnetischen Suszeptiblität als Funktion der Iterationsschritte

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `size`: Anzahl der Gitterplätze je Raumdimension
- `dimension`: Dimension des Gitters
- `temperature`: dimensionslose Temperatur
- `algorithm`: zu verwendender Algorithmus (Metropolis oder Wolff)
- `log_n_steps_init`: Zehnerlogarithmus der Anzahl der Initalisierungsschritte
- `log_n_steps_min_max`: Endpunkte der Zehnerlogarithmen der Anzahl der Iterationsschritte, für die Daten dargestellt werden sollen
- `absmagn`: falls gleich `True` wird statt der Magnetisierung deren Absolutbetrag zur Berechnung herangezogen

Es werden die spezifische Wärme und die magnetische Suszeptibilität jeweils je Spin als Funktion der Iterationsschritte graphisch dargestellt. 

In [ ]:
widget_dict = get_widget_dict()
setattr(widget_dict["log_n_steps_min_max"], "value", (3, 5))

@interact_start(**widget_dict)
def plot_c_chi(size, dimension, temperature, algorithm,
               log_n_steps_init, log_n_steps_min_max,
               absmagn):
    log_n_steps_min, log_n_steps_max = log_n_steps_min_max
    n_spins = size**dimension
    beta = 1 / temperature
    n_iters = logarithmic_niters(log_n_steps_min,
                                 log_n_steps_max, 200)
    thermo_values = thermo_values_development(
        size, dimension, beta, algorithm,
        10**log_n_steps_init, n_iters, absmagn)
    spec_heat = (thermo_values.e2_sum
                 - thermo_values.e_sum**2
                 ) * beta**2 / n_spins
    magn_susc = (thermo_values.m2_sum
                 - thermo_values.m_sum**2
                 ) * beta / n_spins

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    ax1.semilogx(thermo_values.n_steps, spec_heat)
    ax1.set_xlabel("$n$")
    ax1.set_ylabel("$C/Nk$")

    if absmagn:
        label_chi = r"$\chi'/N$"
    else:
        label_chi = r"$\chi/N$"
    ax2.semilogx(thermo_values.n_steps, magn_susc)
    ax2.set_xlabel("$n$")
    ax2.set_ylabel(label_chi)

## Graphische Darstellung der spezifischen Wärme und der magnetischen Suszeptibilität als Funktion der Temperatur

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `size`: Anzahl der Gitterplätze je Raumdimension
- `dimension`: Dimension des Gitters
- `algorithm`: zu verwendender Algorithmus (Metropolis oder Wolff)
- `temperature_range`: Bereich der dimensionslosen Temperatur
- `n_T`: Anzahl der zu betrachtenden Temperaturwerte
- `log_n_steps_init`: Zehnerlogarithmus der Anzahl der Initalisierungsschritte
- `log_n_steps_min_max`: Endpunkte der Zehnerlogarithmen der Anzahl der Iterationsschritte, für die Daten dargestellt werden sollen
- `absmagn`: falls gleich `True` wird statt der Magnetisierung deren Absolutbetrag zur Berechnung herangezogen

Da die Rechnung typischerweise einige Zeit in Anspruch nimmt, werden im Verlauf der Rechnung die berechneten Werte für die Temperatur, die spezifische Wärme und die magnetische Suszeptibilität ausgegeben. Abschließend werden die spezifische Wärme und die magnetische Suszeptibilität jeweils je Spin als Funktion der Temperatur graphisch dargestellt. 

In [ ]:
widget_dict = get_widget_dict()
setattr(widget_dict["algorithm"], "value", "Wolff")

@interact_start(**widget_dict)
def plot_c_chi_of_temp(size, dimension, algorithm,
                       temperature_range, n_T,
                       log_n_steps_init, log_n_steps,
                       absmagn):
    n_spins = size**dimension
    temperature_min, temperature_max = temperature_range
    temperature_vals = np.linspace(temperature_min,
                                   temperature_max, n_T)
    if absmagn:
        headline = "  T      C/N      χ'/N"
        label_chi = r"$\chi'/N$"
    else:
        headline = "  T      C/N      χ/N"
        label_chi = r"$\chi/N$"
    spec_heat_vals = []
    magn_susc_vals = []
    print(headline)
    for temperature in temperature_vals:
        beta = 1 / temperature
        thermo_values = thermo_values_development(
            size, dimension, beta, algorithm,
            10**log_n_steps_init, (10**log_n_steps,),
            absmagn)
        spec_heat = (thermo_values.e2_sum[-1]
                     - thermo_values.e_sum[-1]**2)
        spec_heat = spec_heat * beta**2 / n_spins
        spec_heat_vals.append(spec_heat)

        magn_susc = (thermo_values.m2_sum[-1]
                     - thermo_values.m_sum[-1]**2)
        magn_susc = magn_susc * beta / n_spins
        magn_susc_vals.append(magn_susc)
        print(f"{temperature:6.4f} {spec_heat:7.4f} "
              f"{magn_susc:8.2f}")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.plot(temperature_vals, spec_heat_vals)
    ax1.set_xlabel("$T$")
    ax1.set_ylabel("$C/N$")


    if absmagn:
        ax2.plot(temperature_vals, magn_susc_vals)
    else:
        ax2.semilogy(temperature_vals, magn_susc_vals)
        ax2.semilogy(temperature_vals,
                     n_spins/np.array(temperature_vals))
    ax2.set_xlabel("$T$")
    ax2.set_ylabel(label_chi)